# Populate `zip_code_dict.pickle`

^^ this is the one and only thing this file should do

In [36]:
from shapely.geometry import Polygon, Point
import networkx as nx
import random
from uszipcode import SearchEngine
import pickle
import fiona
from shapely.geometry import shape
from pyproj import Proj, transform, Transformer

g_pickle = "02_boston-area.gpickle"
g = nx.read_gpickle(g_pickle)


## Populate zip_code_dict from CSV file

In [37]:
import csv
g = nx.read_gpickle("02_boston-area.gpickle")

with open('zip_code_dict.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

with open('./data/boston-metro_zipcodes.csv', newline='') as csvfile:
    r = csv.reader(csvfile, delimiter=',')
    for row in r:
        if row[0] in g.nodes().data():
            zip_code_dict[row[0]] = row[1]

with open('zip_code_dict.pickle', 'wb') as handle:
    pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [38]:
shapes = fiona.open("./mass_zips/zipcodes_nt/ZIPCODES_NT_POLY.shp")
input_proj = shapes.crs.get('init')
output_proj = "epsg:4326"
transformer = Transformer.from_crs(input_proj, output_proj)

polygons = {}
for shape in shapes:
    zip_code = shape.get('properties').get('POSTCODE')    
    points = []
    for coord in shape.get('geometry').get('coordinates')[0]:
        x = coord[0]
        y = coord[1]
        xy = transformer.transform(x,y)
        points.append([xy[1], xy[0]])
    try: 
        polygons[zip_code] = Polygon(points)
    except: print(zip_code)

02360
02790
01450
02535
01742
02568
02554
02649
01938
01844
01826
02667
02050
02539
02536
01930
01463
02777
01585
02748
02332
01966
02481
02655
02420
02043
01852
02740
02559
01529
01905
01945
02630
02571
02066
01969
01951
01879
01701
02563
02739
01821
01050
02668
02653
01778
01760
01952
01982
01915
02719
02642
01923
01929
02713
02738
02186
02169
01803
02633
01504
02718
02025
02673
01862
01950
01008
01835
01944
02148
02472
01854
02675
02155
02045
02191
02543
01380
02453
02125
02127
02670
02534
02744
02129
02026
02532
01908
02791
02641
01965
02210
02647
02116
02215
02462
02467
02114
02163
02671
02467
02152


In [32]:
with open('zip_code_dict.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

In [33]:
g_pickle = "02_boston-area.gpickle"
g = nx.read_gpickle(g_pickle)
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in polygons:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break
            
with open('zip_code_dict.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL) 

In [34]:
len(zip_code_dict)

77404

In [35]:
len(g)

96360

In [42]:
len(zip_code_dict)

78142

# Populate zip_code_dict from `uszipcode` library's Polygons

In [41]:
search = SearchEngine(simple_zipcode=False, db_file_dir="./zip_code_cache")
zip_codes = [x for x in polygons]

bad_zips = []
polygons = {}
zip_codes_copy = zip_codes.copy()
for z in zip_codes_copy:
            try: polygons[z] = Polygon(search.by_zipcode(z).polygon)
            except: 
                print(z)
                if z not in bad_zips:
                    bad_zips.append(z)
                zip_codes.remove(z)

with open('zip_code_dict_NEW.pickle', 'rb') as handle:
    zip_code_dict = pickle.load(handle)

for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    lon,lat = node_data[1]['lon'], node_data[1]['lat']
    if node_id in zip_code_dict:
        continue
    for z in zip_codes:
        polygon = polygons[z]
        if polygon.contains(Point(lon,lat)):
            zip_code_dict[node_id] = z
            break
            
with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)            
        

zip_code_dict
        
unprocessed = []
for node_data in g.nodes().data():
    node_id = node_data[1]["id"]
    if node_id not in zip_code_dict:
        unprocessed.append(node_id)
print("Number still unprocessed: " + str(len(unprocessed)))
print("Add the following to the zip_codes set:")

01230
01201
01590
01096
01057
01083
01253
01225
01011
01440
01258
01604
01349
01032
02638
02482
01226
01062
01757
01772
01262
01020
02745
01534
02660
02130
01588
02639
02452
01611
02131
02445
01902
02122
01069
02071
01260
02464
02663
02108
01745
01438
01063
01561
02723
Number still unprocessed: 18218
Add the following to the zip_codes set:


In [45]:
for polygon in polygons:
    if polygons.get(polygon).contains(Point([-71.1191928, 42.4057])):
        print(polygon)

print(search.by_coordinates(42.4057, -71.1191928, returns=1)[0].zipcode)

# print(polygons.get('02143').contains(Point([-71.1191928, 42.4057])))

02144


In [49]:
len_g = len(g)

In [50]:
def get_zip_code_dict():
    g = nx.read_gpickle("02_boston-area.gpickle")
    nodes_data_subset = g.nodes().data()
    count = 1
    with open('zip_code_dict_NEW.pickle', 'rb') as handle:
        zip_code_dict = pickle.load(handle)
   

    for node_data in g.nodes().data():
        node_id = node_data[1]["id"]
        lon,lat = node_data[1]['lon'], node_data[1]['lat']
        if node_id not in zip_code_dict:
            count += 1
            zip_code_dict[node_id] = search.by_coordinates(lat, lon, returns=1)[0].zipcode
            print(zip_code_dict[node_id])
            print(str(len(zip_code_dict)) + "       "+ str(len(zip_code_dict)/len_g) + "        ...slow algo alert")


        if count % 25 == 0: 
            print("--> Dumping into zip_code_dict.pickle")
            with open('zip_code_dict_NEW.pickle', 'wb') as handle:
                pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        

    with open('zip_code_dict_NEW.pickle', 'wb') as handle:
        pickle.dump(zip_code_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return zip_code_dict

In [ ]:
get_zip_code_dict()

02144
78143       0.8109485263594852        ...slow algo alert
02144
78144       0.810958904109589        ...slow algo alert
02144
78145       0.8109692818596929        ...slow algo alert
02114
78146       0.8109796596097966        ...slow algo alert
02114
78147       0.8109900373599004        ...slow algo alert
02114
78148       0.8110004151100042        ...slow algo alert
02118
78149       0.8110107928601079        ...slow algo alert
02118
78150       0.8110211706102117        ...slow algo alert
02118
78151       0.8110315483603154        ...slow algo alert
02118
78152       0.8110419261104193        ...slow algo alert
02210
78153       0.8110523038605231        ...slow algo alert
02210
78154       0.8110626816106268        ...slow algo alert
02210
78155       0.8110730593607306        ...slow algo alert
02210
78156       0.8110834371108344        ...slow algo alert
02210
78157       0.8110938148609381        ...slow algo alert
02129
78158       0.8111041926110419        ...slow algo

02129
78224       0.8117891241178913        ...slow algo alert
02129
78225       0.811799501867995        ...slow algo alert
02129
78226       0.8118098796180988        ...slow algo alert
02108
78227       0.8118202573682026        ...slow algo alert
02108
78228       0.8118306351183063        ...slow algo alert
02108
78229       0.8118410128684102        ...slow algo alert
02129
78230       0.8118513906185139        ...slow algo alert
02127
78231       0.8118617683686177        ...slow algo alert
02116
78232       0.8118721461187215        ...slow algo alert
02116
78233       0.8118825238688252        ...slow algo alert
02116
78234       0.811892901618929        ...slow algo alert
02116
78235       0.8119032793690328        ...slow algo alert
02116
78236       0.8119136571191365        ...slow algo alert
02116
78237       0.8119240348692404        ...slow algo alert
02115
78238       0.8119344126193442        ...slow algo alert
02115
78239       0.8119447903694479        ...slow algo 

02114
78332       0.8129099211290992        ...slow algo alert
02115
78333       0.812920298879203        ...slow algo alert
02115
78334       0.8129306766293067        ...slow algo alert
02115
78335       0.8129410543794106        ...slow algo alert
02115
78336       0.8129514321295144        ...slow algo alert
02115
78337       0.8129618098796181        ...slow algo alert
02115
78338       0.8129721876297219        ...slow algo alert
02115
78339       0.8129825653798256        ...slow algo alert
02115
78340       0.8129929431299294        ...slow algo alert
02108
78341       0.8130033208800332        ...slow algo alert
--> Dumping into zip_code_dict.pickle
02108
78342       0.813013698630137        ...slow algo alert
02108
78343       0.8130240763802408        ...slow algo alert
02114
78344       0.8130344541303446        ...slow algo alert
02114
78345       0.8130448318804483        ...slow algo alert
02114
78346       0.8130552096305521        ...slow algo alert
02114
78347       0

02111
78433       0.8139580738895807        ...slow algo alert
02118
78434       0.8139684516396846        ...slow algo alert
02127
78435       0.8139788293897883        ...slow algo alert
02127
78436       0.8139892071398921        ...slow algo alert
02127
78437       0.8139995848899958        ...slow algo alert
02121
78438       0.8140099626400996        ...slow algo alert
02121
78439       0.8140203403902034        ...slow algo alert
02129
78440       0.8140307181403071        ...slow algo alert
02129
78441       0.814041095890411        ...slow algo alert
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into zip_code_dict.pickle
--> Dumping into